**Sprint ディープラーニングフレームワーク1**

# 【問題1】スクラッチを振り返る

* ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙しろ。

**【解答】**

1. 入力データの準備
2. テストデータと訓練データに分ける。
3. 入力データ内から、ランダムに「ミニバッチ」を取り出す
4. 「バイアス」と「重み」を初期化する。
5. 順伝搬を行う
    1. 訓練データをパッド分に展開する。
    2. 訓練データをフィルター（重み）に通して乗算する。
    3. バイアスを加算する。
    4. 活性化関数を行う
    5. 全層を通過後、出力の際softmax関数を用いて、値を正規化する。
    6. 出力した値を記録して、次のミニバッチの学習を行う。
6. 逆伝搬を行う
    1. 各重みパラメータに関する損失関数の勾配を求める

7. 重みパラメータを勾配方向に微小量だけ更新する。

# 【問題2】スクラッチとTensorFlowの対応を考える

In [6]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
# print(y, y.shape, type(y))
y = y.astype(np.int)[:, np.newaxis]
# print(y, y.shape, type(y))
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 20
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)    
        total_loss = np.empty(total_batch)
        # total_loss = 0
        total_acc = np.empty(total_batch)
        # total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_loss.mean()
        #total_loss /= batch_size
        total_acc /= total_acc.mean()
        #total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.8f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 7.44431210, val_loss : 12.3300, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 4.21767378, val_loss : 4.9650, acc : 0.750, val_acc : 0.625
Epoch 2, loss : 13.51782227, val_loss : 3.4740, acc : 0.500, val_acc : 0.812
Epoch 3, loss : 4.26804495, val_loss : 5.2629, acc : 0.750, val_acc : 0.562
Epoch 4, loss : 5.67510748, val_loss : 2.3903, acc : 0.500, val_acc : 0.750
Epoch 5, loss : 0.00879623, val_loss : 0.7658, acc : 1.000, val_acc : 0.875
Epoch 6, loss : 0.00003815, val_loss : 0.0002, acc : 1.000, val_acc : 1.000
Epoch 7, loss : 0.00000000, val_loss : 0.0039, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.00000000, val_loss : 0.0001, acc : 1.000, val_acc : 1.000
Epoch 9, loss : 0.00000000, val_loss : 0.0013, acc : 1.000, val_acc : 1.000
Epoch 10, loss : 0.00000000, val_loss : 0.0003, acc : 1.000, val_acc : 1.000
Epoch 11, loss : 0.00000000, val_loss : 0.0185, acc : 1.000, val_acc : 1.000
Epoch 12, loss : 0.00000000, val_loss : 0.1414, acc : 1.000, val_acc : 0.938
Epoch 1